In [1]:
import string
import nltk
import csv
from collections import Counter
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer

In [2]:
def tokenize_text(filename):
    """
    This function generates a list of tokens with punctuation
    stopwords, and spaces removed for the whole text.
    """
    text_tokens = []

    file_path = "./corpora/" + filename
    stop = list(set(stopwords.words('english')))
    punc = [p for p in string.punctuation]
    stop_punc = stop + punc + ["''", '``']
    text = open(file_path, 'r')

    for row in text:
        tokens = word_tokenize(row)# splits string
        if len(tokens) is not 0:
            # puts everything in lowercase, removes punctuation and stopwords
            tokens = [token.lower() for token in tokens if token not in stop_punc]
            # adds row tokens to master list
            text_tokens.extend(tokens)

    return text_tokens

In [3]:
tokenized = tokenize_text('Leroux_ThePhantomOfTheOpera_Gutenberg.txt')

In [4]:
def get_color_words():
    """
    Gets color words from the csv file and puts them into a dict where key = word
    and value = hex value.
    """
    color_word_dict = {}
    color_data = csv.reader(open('./color_names.csv'), delimiter=",", quotechar='"')

    for row in color_data:
        if row[0] != "Colour Name":
            name = row[0].lower()
            family = row[2].lower()
            hex_value = row[3].lower()
            color_word_dict[name] = (hex_value, family)
    return color_word_dict

In [5]:
color_dict = get_color_words()
print("There are %d color words in our dictionary." % len(color_dict))
print()
sample_colors = list(color_dict.keys())[300:320]
print("Here is a sample - first value is the HTML color, the second is the color family.")
for s in sample_colors:
    print(s, ":", color_dict[s])

There are 1095 color words in our dictionary.

Here is a sample - first value is the HTML color, the second is the color family.
rush : ('darkred', 'brownish-red')
brickdust-like : ('darkred', 'brownish-red')
brickdusty : ('darkred', 'brownish-red')
chestnut-red : ('darkred', 'brownish-red')
terra-cotta : ('darkred', 'brownish-red')
copper-red : ('darkred', 'brownish-red')
bricky : ('darkred', 'brownish-red')
brickish : ('darkred', 'brownish-red')
fox-red : ('darkred', 'brownish-red')
oxblood : ('darkred', 'brownish-red')
tony : ('darkred', 'brownish-red')
mahogany-red : ('darkred', 'brownish-red')
roan : ('lightsalmon', 'yellowish-red')
sherry-red : ('lightsalmon', 'yellowish-red')
port-winey : ('lightsalmon', 'yellowish-red')
fire-red : ('orangered', 'orange-red')
flame-red : ('orangered', 'orange-red')
flammid : ('orangered', 'orange-red')
flammeous : ('orangered', 'orange-red')
pimento : ('orangered', 'orange-red')
